In [1]:
import pennylane as qml
from pennylane import numpy as np

In [2]:
def triple_excitation_matrix(gamma):
    """The matrix representation of a triple-excitation Givens rotation.

    Args:
        - gamma (float): The angle of rotation

    Returns:
        - (np.ndarray): The matrix representation of a triple-excitation
    """

    # QHACK #
    # Generate coefficients
    c = qml.math.cos(gamma / 2)
    s = qml.math.sin(gamma / 2)
    # Start with a diagonal matrix
    mat = qml.math.diag([1.0] * (2 ** NUM_WIRES))
    # Decimal representations of states
    i, j = 7, 56
    # Alter matrix values that alter only the states 7 and 56
    mat[i, i] = c
    mat[i, j] = -s
    mat[j, i] = s
    mat[j, j] = c
    return mat

In [3]:
dev = qml.device("default.qubit", wires=6)
NUM_WIRES = 6

@qml.qnode(dev)
def circuit(angles):
    """Prepares the quantum state in the problem statement and returns qml.probs

    Args:
        - angles (list(float)): The relevant angles in the problem statement in this order:
        [alpha, beta, gamma]

    Returns:
        - (np.tensor): The probability of each computational basis state
    """
    # QHACK #
    alpha, beta, gamma = angles[0], angles[1], angles[2]
    # Initialize state
    qml.BasisState(np.array([1, 1, 1, 0, 0, 0]), wires=[0, 1, 2, 3, 4, 5])
    # SingleExcitation gate
    qml.SingleExcitation(alpha, wires=[0, 5])
    # DoubleExcitation gate
    qml.DoubleExcitation(beta, wires=[0, 1, 4, 5])
    # TripleExcitation gate from our implementation
    qml.QubitUnitary(triple_excitation_matrix(gamma), wires=[0, 1, 2, 3, 4, 5])
    # QHACK #
    return qml.probs(wires=range(NUM_WIRES))

In [4]:
inputs = "2.71035258,2.86582337,4.46182774".split(",")
inputs = np.array(inputs, dtype=float)
probs = circuit(inputs).round(6)

# We interested in the states |111000> = 56, |000111> = 7, |001011> = 11 and |011001> = 25
a, b, g = inputs[0], inputs[1], inputs[2]
p_56 = probs[56]
p_7 = probs[7]
p_11 = probs[11]
p_25 = probs[25]
print(f"Measured prob |111000> {p_56} vs expected {(np.cos(a/2)*np.cos(b/2)*np.cos(g/2))**2}")
print(f"Measured prob |000111> {p_7} vs expected {(np.cos(a/2)*np.cos(b/2)*np.sin(g/2))**2}")
print(f"Measured prob |001011> {p_11} vs expected {(np.cos(a/2)*np.sin(b/2))**2}")
print(f"Measured prob |011001> {p_25} vs expected {(np.sin(a/2))**2}")

Measured prob |111000> 0.000325 vs expected 0.0003251869843297567
Measured prob |000111> 0.00054 vs expected 0.0005396118866232373
Measured prob |001011> 0.044911 vs expected 0.04491115090944501
Measured prob |011001> 0.954224 vs expected 0.9542240502196023


In [5]:
inputs = (np.random.rand(3)*2-1)*2*np.pi
probs = circuit(inputs).round(6)

# We interested in the states |111000> = 56, |000111> = 7, |001011> = 11 and |011001> = 25
a, b, g = inputs[0], inputs[1], inputs[2]
p_56 = probs[56]
p_7 = probs[7]
p_11 = probs[11]
p_25 = probs[25]
print(f"Measured prob |111000> {p_56} vs expected {(np.cos(a/2)*np.cos(b/2)*np.cos(g/2))**2}")
print(f"Measured prob |000111> {p_7} vs expected {(np.cos(a/2)*np.cos(b/2)*np.sin(g/2))**2}")
print(f"Measured prob |001011> {p_11} vs expected {(np.cos(a/2)*np.sin(b/2))**2}")
print(f"Measured prob |011001> {p_25} vs expected {(np.sin(a/2))**2}")

Measured prob |111000> 0.043423 vs expected 0.04342292706512467
Measured prob |000111> 0.179567 vs expected 0.17956680677628428
Measured prob |001011> 0.463535 vs expected 0.4635351746666425
Measured prob |011001> 0.313475 vs expected 0.31347509149194847
